## ¿Sirve o no sirve buscar precios de propiedades en campo description a traves de regex?
---

### Importo, preparo y cargo df

In [32]:
import pandas as pd
import numpy as np
import re

In [33]:
df_propierati = pd.read_csv('../properati.zip', compression='zip')
df_prop_uz = pd.read_csv('../properati.csv')

In [34]:
pd.set_option('display.max_colwidth', 1500)
pd.set_option('display.max_rows', 200)
#pd.set_option('precision', 5)

### Preguntas iniciales

Nulos en descripcion. Muy pocos. Parece prometedor.

In [35]:
df_propierati.description.isnull().value_counts()

False    121218
True          2
Name: description, dtype: int64

Extraigo valores de descripcion para buscar en regex101
(patron de busqueda ejemplo en: https://regex101.com/r/EhnQBv/1)

In [36]:
#salida_desc = df_propierati.description.tail(200)
#salida_desc.to_csv('descripcion_sample.txt', index=False)

#salida_titl = df_propierati.title.tail(200)
#salida_titl.to_csv('titl_sample.txt', index=False)

#df_propierati.loc[121020:121035, 'state_name':]

### Trabajo con expresiones regulares y mascaras booleanas.

1. Creo patron regex y lo compilo

 Patron: `([uU]\$?[sSdD\$]?[dD\$]?|dolares)\s?(\d+\.?\,?\d+\.?\,?\d+\.?\,?\d{1,}\,?\.?\d+)(?!(.*m2|.*mc|.*metros))`<br>
 
- Comienza identificando la moneda (USD, usd, U\\$D, US\\$, etc.)<br>
- Valor: desde 5 caracteres numericos, identifica comas (,) y puntos (.)<br>
- Excluye al final caracteres que hagan referencia a metros cuadrados para evitar esos valores.<br>

2. Creo objeto de regex en nueva columna matches_precio para idenficar las coincidencias del pattron en la columna description

3. Para identificar la efectividad y el valor de asignar precio desde la descripcion en el campo `price_aprox_usd` creo mascara booleana:

- valores nulos de columna `price_aprox_usd` <br>
- valores no nulos de columna `matches_precio`<br>
- AND: mascara booleana con todos los valores nulos de `price_aprox_usd` que si tengan un valor identificado en el campo `matches_precio` (viene de `description`). Resultado: 583 registros<br>

In [49]:
# regex - key insensitive
precio_dolares_patt = '(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)\d{0,3}\,?\.?\d{0,3}\,?\.?\d{0,3}) ?(?!(.*m2|.*mc|.*metros))'
precio_dolares_regex = re.compile(precio_dolares_patt, flags = re.IGNORECASE)

# objetos regex en nueva columna
df_propierati['matches_precio_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else precio_dolares_regex.search(x))

# objetos regex en nueva columna
df_propierati['matches_precio_titl'] = df_propierati.title.apply(lambda x: x if x is np.NaN else precio_dolares_regex.search(x))

# mascara booleana - valores nulos en price_aprox_usd que SI puede ser completados desde el regex sobre description
price_aprox_usd_mask = df_propierati.price_aprox_usd.isnull()
# descripcion
matches_precio_desc_mask = df_propierati.matches_precio_desc.notnull()
# titulo
matches_precio_titl_mask = df_propierati.matches_precio_titl.notnull()
#
precio_desc_si_price_usd_no_mask = matches_precio_desc_mask & price_aprox_usd_mask

precio_titl_si_price_usd_no_mask = matches_precio_titl_mask & price_aprox_usd_mask

print('descripcion:', precio_desc_si_price_usd_no_mask.value_counts())
print('titulo:', precio_titl_si_price_usd_no_mask.value_counts())

# Comprobacion
# 583 registros
# df_propierati.loc[precio_desc_si_price_usd_no_mask, ['matches_precio','price_aprox_usd']].shape[0]

# Extraccion de muestra -- descripcion
df_propierati.loc[precio_desc_si_price_usd_no_mask, ['matches_precio_desc','price_aprox_usd']].tail(200)

# Extraccion de muestra -- titulo
df_propierati.loc[precio_titl_si_price_usd_no_mask, ['matches_precio_titl','price_aprox_usd']].tail(200)

descripcion: False    120667
True        553
dtype: int64
titulo: False    118835
True       2385
dtype: int64


,matches_precio_titl,price_aprox_usd
4074,"<re.Match object; span=(0, 12), match='U$D 235.000 '>",NaN
4075,"<re.Match object; span=(0, 12), match='U$D 137.500 '>",NaN
4076,"<re.Match object; span=(0, 12), match='U$D 550.000 '>",NaN
4077,"<re.Match object; span=(0, 12), match='U$D 200.000 '>",NaN
4078,"<re.Match object; span=(0, 12), match='U$D 350.000 '>",NaN
4079,"<re.Match object; span=(0, 12), match='U$D 475.000 '>",NaN
4080,"<re.Match object; span=(0, 12), match='U$D 310.000 '>",NaN
4081,"<re.Match object; span=(0, 12), match='U$D 225.000 '>",NaN
4082,"<re.Match object; span=(0, 12), match='U$D 595.000 '>",NaN
4083,"<re.Match object; span=(0, 12), match='U$D 280.000 '>",NaN


Falta depurar expresion regular:
- se estan agregando los precios de las cocheras.
- hay valores de precios no finales. Por ejemplo se observan valores de adelantos mas financiacion.

Ejemplos de valores repetidos:

In [38]:
df_propierati.description[120910:120914]

120910                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR WENGUE- AMPLIO BAÑO COMPLETO REVESTIDO CON CERÁMICA SAN LORENZO COLOR ARENA Y GUARDAS DE VENECITAS- ARTEFACTOS SANITARIOS Y GRIFERÍAS SERÁN DE MARCAS DE PRIMERA LÍNEA (ROCA, GENEBRE, FERRUM O SIMILAR)- PLACARD COMPLETO-COCINA CON MESADA DE GRANITO, MUEBLES BAJO Y SOBRE MESADA COMPLETOS-  COCINA CON HORNO Y ANAFE DE 4 HORNALLAS A GAS- PRE-INSTALACIÓN PARA EQUIPOS DE AIRE ACONDICIONADOAGUA CALIENTE CENTRAL-BALCÓN CON CONEXIÓN PARA LAVARROPAS- AMENITIES: SOLÁRIUM CON PARRILLA.ENTREGA SEPTIEMBRE 2018PRECIO DE CONTADO- CONSULTAR POR FINANCIACIÓN ADELANTO DE 35% Y CUOTAS CON AJUSTE CAC.COCHERAS OPCIONALES DESDE u$s20.000.-BAULERAS OPCIONALES u$S2.200.-CONSÚLTENOS- SUCURSAL PALERMO-             
120911                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR WENGUE- AMPLIO BAÑO COMPLETO REVESTIDO CON CERÁMICA SAN LORENZO COLOR ARENA Y GUARDAS DE V

In [39]:
# muchos nulos de precios en dolares
df_propierati.price_aprox_usd.isnull().value_counts()

False    100810
True      20410
Name: price_aprox_usd, dtype: int64

In [40]:
# aparecen 11000 registros de precios en la descripcion
df_propierati.matches_precio.notnull().value_counts()

False    111284
True       9936
Name: matches_precio, dtype: int64

In [41]:
(df_propierati.price_aprox_usd.isnull() & df_propierati.matches_precio.notnull()).value_counts()

False    120584
True        636
dtype: int64

In [42]:
(df_propierati.price_aprox_usd == 0).value_counts()

False    121219
True          1
Name: price_aprox_usd, dtype: int64

In [53]:
df_propierati.loc[4206]

Unnamed: 0                                                                                                                                                                                                                                                                   4206
operation                                                                                                                                                                                                                                                                    sell
property_type                                                                                                                                                                                                                                                               house
place_name                                                                                                                                                                        